# Always

In [ ]:
%pylab inline

In [ ]:
plt.rc('font',family='Helvetica')
plt.rcParams["font.size"] = 20

## Set nbname and wrkdir

In [ ]:
nbname=''
wrkdir=''

# Matplotlib

## Normal plot

In [ ]:
fig0 = plt.figure(figsize=(7,7))
ax0 = fig0.add_subplot(111)
ax0.plot(f,-z,'o',label='f')
ax0.set_ylim(-5500,0)
ax0.set_xlim(0,4)
ax0.set_xlabel('f')
ax0.set_ylabel('depth')
ax0.legend(loc='lower right',ncol=1)
ax0.grid()
fig0.tight_layout()
savefig('../figs/C_fesed_CESM-Std.png',dpi=200,bbox_inches='tight')

## Basemap

In [ ]:
fig=plt.figure(figsize=(15,10))

lat_mn=-90.0
lat_mx=90.0
lon_mn=0.0
lon_mx=360.0

for n in range(12):
    ax=fig.add_subplot(4,3,n+1)
    # ax.set_title('hoge',loc='left')
    m = Basemap(projection='cyl',llcrnrlat=lat_mn,urcrnrlat=lat_mx,llcrnrlon=lon_mn,urcrnrlon=lon_mx,resolution='l')
    x,y = m(g0.variables['vlon'],g0.variables['vlat'])
    im = m.pcolormesh(x,y,v[n,:,:],vmin=-13.0,vmax=-9.0)
    m.drawcoastlines()
    n=n+1
fig.subplots_adjust(bottom=0.05)
cax=fig.add_axes([0.2,0.01,0.6,0.03]) # x_lower_left, y_lower_left, width, height
cb=fig.colorbar(im,cax=cax,orientation='horizontal')
# cb=m.colorbar() # if colorbar to be fitted on maps
plt.tight_layout() # note that tight layout sometimes conflicts with subplots_adjust

plt.savefig('figs/fig.png',dpi=300,bbox_inches='tight')

## Custom color levels

In [ ]:
levels = ([3.2,5.6,10.0,18.0,32.0,56.0,100.0,180.0,320.0,560.0,1000.0,1800.0,3200.0,5600.0,10000.0])
norm = BoundaryNorm(levels, ncolors=cm.N, clip=True)
im0 = m0.pcolormesh(x,y,cs137s[cnt,:,:], norm=norm,cmap=cm)

## Define colormap

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
def generate_cmap(colors):
    values = range(len(colors))
    vmax = np.ceil(np.max(values))
    color_list = []
    for v, c in zip(values, colors):
        color_list.append( ( v/ vmax, c) )
    return LinearSegmentedColormap.from_list('custom_cmap', color_list)

### Usage

In [ ]:
cm = generate_cmap(['#ffffff','#6385c5','#6bad40','#eee959','#c04d29'])

## Modification of colorbar

In [ ]:
cax = fig0.add_axes([0.557, 0.52, 0.02, 0.3])
cbar = fig0.colorbar(im0, cax=cax, ticks=[3.2,10.0,32.0,100.0,320.0,1000.0,3200.0,10000.0])
cbar.ax.set_yticklabels([3.2,10,32,100,320,1000,3200,10000])

## Add mark or text

In [ ]:
xx, yy = m0(lon0,lat0)
m0.plot(xx,yy,'ko',ms=10)

In [ ]:
tx,ty = m0(140.1,39.5)
ax0.text(tx,ty,'${}^{137}\mathrm{Cs_{wat}} \:\: \mathrm{Bq \: m^{-3}}$',va='top',ha='left',fontsize=32)

## Plot POP data

In [ ]:
lon2d = np.where(np.greater_equal(lon2d,min(lon2d[:,0])),lon2d-360,lon2d)
ep_gx1v6_2=ma.concatenate((ep_gx1v6,ep_gx1v6),axis=2)
lon2d_2=ma.concatenate((lon2d,lon2d+360.),axis=1)
lat2d_2=ma.concatenate((lat2d,lat2d),axis=1)

In [ ]:
src_nc00=netCDF4.Dataset('/Users/misumi/pop_data/gx1v6.concat.nc','r')
tlon=src_nc00.variables['tlon'][:]
tlat=src_nc00.variables['tlat'][:]
vlon=src_nc00.variables['vlon'][:]
vlat=src_nc00.variables['vlat'][:]
src_nc00.close()

var2=ma.concatenate((var,var),axis=1)

fig0 = plt.figure(figsize=(20,10))

lon_mn = 0.0
lon_mx = 360.0
lat_mn = -90.0
lat_mx = 90.0

ax0 = fig0.add_subplot(111)
m0 = Basemap(projection='cyl',llcrnrlat=lat_mn,urcrnrlat=lat_mx,llcrnrlon=lon_mn,urcrnrlon=lon_mx,resolution='c')
x,y = m0(vlon,vlat)
im0 = m0.pcolormesh(x,y,var2)
m0.drawcoastlines()
cb0 = m0.colorbar(im0)

plt.tight_layout()

# Pyroms

## Grid

An example of gridid.txt is in /Users/misumi/roms_data.

In [ ]:
import os
os.environ["PYROMS_GRIDID_FILE"] = "gridid.txt"

In [ ]:
grd_h025 = pyroms.grid.get_ROMS_grid('h025_l045_v001')

## Remapping

To make the grid infomation files, please refer to work/remap_tools.

In [ ]:
pyroms.remapping.make_remap_grid_file(grd_h025)

In [ ]:
pyroms.remapping.compute_remap_weights(\
     grid1_file='../remap/remap_grid_poc_rho.nc',\
     grid2_file='../remap/remap_grid_h025_l045_v003_rho.nc',\
     interp_file1='../remap/remap_wgt_poc_to_h025_l045_v003_rho.nc',\
     interp_file2='N/A', \
     map1_name='poc to h025_l045_v002 bilinear', \
     map2_name='N/A', \
     num_maps='1',\
     map_method='bilinear',\
     luse_grid1_area='.false.',\
     luse_grid2_area='.false.',\
     normalize_opt='fracarea',\
     output_opt='scrip',\
     restrict_type='latitude',\
     num_srch_bins='90',\
     grid1_periodic='.false.',\
     grid2_periodic='.false' )

In [ ]:
frac = pyroms.remapping.remap(topo_l[2100:4800,3000:9000],'../remap/remap_wgt_etopo2v2c_to_h025_l045_v002_rho.nc')

# netCDF

## write

|id     |meaning                |
|:------|:----------------------|
|'f4'   |32-bit floating point  |
|'f8'   |64-bit floating point  |
|'i2'   |16-bit signed integer  |
|'i4'   |32-bit signed integer  |
|'i8'   |64-bit signed integer  |
|'u1'   |8-bit unsigned integer |
|'u2'   |16-bit unsigned integer|
|'u4'   |32-bit unsigned integer|
|'u8'   |64-bit unsigned integer|
|'S1'   |single character string|

In [ ]:
fo0=netCDF4.Dataset('data/vgpm.ep.nc','w')
fo0.nbname=nbname
fo0.wrkdir=wrkdir

fo0.createDimension('ntime',nt)
fo0.createDimension('nlat' ,ny)
fo0.createDimension('nlon' ,nx)

var0_o=fo0.createVariable('lon' ,'f8',('nlat','nlon'))
var1_o=fo0.createVariable('lat' ,'f8',('nlat','nlon'))
var2_o=fo0.createVariable('ep','f8',('ntime','nlat','nlon'),fill_value=np.nan)
var2_o.units='mmol/m2/d'
var3_o=fo0.createVariable('ep_org','f8',('ntime','nlat','nlon'),fill_value=np.nan)
var3_o.units='mmol/m2/d'

var0_o[:]=lon[:]
var1_o[:]=lat[:]
var2_o[:]=ep[:]
var3_o[:]=ep_org[:]

fo0.close()

## Copy dims and vars

In [ ]:
src_nc01 = netCDF4.Dataset('../data/h025_l045_v001.ini.161108.nc','r')
dst_nc00 = netCDF4.Dataset('../data/test.nc','w')
vnames = src_nc01.variables.keys()
dnames = src_nc01.dimensions.keys()

for d in dnames:
    x = src_nc01.dimensions.get(d)
    dst_nc00.createDimension(x.name,x.size)
    
fv = -999.0
for v in vnames:
    x = src_nc01.variables[v]
    y = dst_nc00.createVariable(x.name,np.dtype('float').char,(x.dimensions),fill_value=fv)

dst_nc00.close()

# pandas

In [ ]:
fe_obs = pd.read_csv('../data/fe_obs.csv')

In [ ]:
fe_obs.to_csv('../data/fe_obs_model.csv',header=True,index=False,encoding='utf-8')

# Datetime object

In [ ]:
date0 = datetime.datetime(2011,3,1)
for n in range(3,30):
    dd = date0 + datetime.timedelta(n)
    print dd.strftime('%Y/%m/%d')

# Format

In [ ]:
print "{0:<10}".format("Hello") #10文字幅。左寄せ
print "{0:^10}".format("Hello") #10文字幅。センタリング
print "{0:>10}".format("Hello") #10文字幅。右寄せ
print "{0:_>10}".format("Hello") # 詰め文字に_を指定。10文字幅。右寄せ
print "{0:0>6}".format(123) #前ゼロ埋め

In [ ]:
print "{0:,d},{1:.3f}".format(1234567,12.34)

In [ ]:
print "int: {0:d};  hex: {0:X};  oct: {0:o};  bin: {0:b}".format(42)

In [ ]:
import datetime
d = datetime.datetime.now()
"{0:%Y-%m-%d %H:%M:%S}".format(d)

# Multi processing

In [ ]:
def func(x,arg0,arg1):
    return x+arg0+arg1

In [ ]:
import concurrent.futures

exe=concurrent.futures.ProcessPoolExecutor(max_workers=16) # Generate instace of executor
xlist=[1.0,2.0,3.0]
arg0=1.0
arg1=2.0
mpj={exe.submit(func,x,arg0,arg1): x for x in xlist}       # Mapping jobs, mpj is a dictionary mpj{exe_obj:input_var}
out={mpj[j]:j.result() for j in concurrent.futures.as_completed(mpj)} # Storing out{input_var:result}

# f2py 

In [ ]:
import sys
sys.path.append('./fortran')

## fortran (calc_sub.f)

In [ ]:
      subroutine calc(b,a,n)
      implicit none
c
      real*8  b(n)
      real*8  a(n)
      integer n, i

Cf2py intent(out) b
Cf2py intent(in)  a
Cf2py intent(in)  n

      do i = 1, n
        b(i) = a(i) + 1.0
      end do
c
      return
      end

## python

In [ ]:
import os
import numpy as np
import numpy.ma as ma
import netCDF4
from pandas import Series, DataFrame
import pandas as pd

import calc_sub

a = np.random.rand(10000)

b = calc_sub.calc(a,10000)

for n in range(10000):
  print a[n], b[n]

# for libs

In [ ]:
import sys
def zCal(grd,pos):
    gh=grd.hgrid
    gv=grd.vgrid
    ny,nx=gv.h.shape
    if   pos=='rho':
        ns=gv.N
        s=gv.s_rho
        c=gv.Cs_r
        m=gh.mask_rho
    elif pos=='w':
        ns=gv.N+1
        s=gv.s_w
        c=gv.Cs_w
        m=gh.mask_rho
    else:
        sys.exit()
    ary=np.zeros((ns,ny,nx))
    hc=ary+gv.hc
    s=ary.swapaxes(0,2)+s
    s=s.swapaxes(0,2)
    h=ary+gv.h
    c=ary.swapaxes(0,2)+c
    c=c.swapaxes(0,2)
    S=(hc*s+h*c)/(hc+h)
    z=S*h
    m=ary+m
    z=ma.masked_where(m==0,z)
    return z